<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Creación-de-la-Clase" data-toc-modified-id="Creación-de-la-Clase-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Creación de la Clase</a></span><ul class="toc-item"><li><span><a href="#Definición-de-las-variables" data-toc-modified-id="Definición-de-las-variables-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Definición de las variables</a></span><ul class="toc-item"><li><span><a href="#Queries-de-las-tablas" data-toc-modified-id="Queries-de-las-tablas-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span><em>Queries</em> de las tablas</a></span></li></ul></li><li><span><a href="#Llamamos-a-la-Clase" data-toc-modified-id="Llamamos-a-la-Clase-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Llamamos a la Clase</a></span><ul class="toc-item"><li><span><a href="#Creamos-la-BBDD" data-toc-modified-id="Creamos-la-BBDD-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Creamos la BBDD</a></span></li><li><span><a href="#Creamos-las-tablas" data-toc-modified-id="Creamos-las-tablas-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Creamos las tablas</a></span></li><li><span><a href="#Insertamos-los-datos" data-toc-modified-id="Insertamos-los-datos-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Insertamos los datos</a></span></li></ul></li></ul></li></ul></div>

En este jupyter vamos a hacer lo mismo que en el anterior, pero en este caso lo haremos definiendo una clase que nos permita hacer todo el proceso de creación e insercción de datos en la BBDD. 

In [1]:
import pandas as pd
import mysql.connector

In [2]:
# importamos el dataframe

df = pd.read_pickle('datos/datos_actualizados.pkl')
df.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,3,9,6,none,0,10,65,cloudynight,N,2,2022-03-16,40.4165,-3.70256,Madrid,2.458333,7.0
1,6,9,6,none,1,10,70,cloudynight,N,3,2022-03-16,40.4165,-3.70256,Madrid,2.458333,7.0


# Creación de la Clase

In [3]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        # nuestra clase va a recibir dos parámetros que son fijos a lo largo de toda la BBDD, el nombre de la BBDD y la contraseña con el servidor. 
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña

    # método para crear la BBDD 
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La BBDD ya existe")
            
    # método para crear tablas  e insertar datos en ellas   
    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            

    # método para comprobar si la ciudad que estamos intentando meter ya existe en la BBDD. 
    # En caso de que ya exista no será necesario que la insertemos de nuevo
    def check_ciudades(self):
    
        mydb = mysql.connector.connect(user='root',
                                      password=f"{self.contraseña}",
                                      host='127.0.0.1',
                                      database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        # query para extraer los valores únicos de ciudades de la tabla de localidades 
        query_existe_ciudad = f"""
                SELECT DISTINCT ciudad FROM localidades
                """
        mycursor.execute(query_existe_ciudad)
        ciudades = mycursor.fetchall()
        return ciudades
    
    # método para sacar el id de una ciudad en concreto 
    def sacar_id_ciudad(self, ciudad):
        
        mydb = mysql.connector.connect(user='root',
                                       password= f'{self.contraseña}',
                                       host='127.0.0.1', 
                                       database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()
        
        try:
            query_sacar_id = f"SELECT idlocalidad FROM localidades WHERE ciudad = '{ciudad}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
            return "Sorry, no tenemos esa ciudad en la BBDD y por lo tanto no te podemos dar su id. "
     
    
    def sacar_id_fecha(self, timepoint, fecha):
        mydb = mysql.connector.connect(user='root', password=f'{self.contraseña}',
                                          host='127.0.0.1', database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        try:
            query_sacar_id = f"SELECT idtiempo FROM tiempo WHERE timepoint = {timepoint} and fecha = '{fecha}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
             return "Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. "
            

## Definición de las variables 

###  *Queries* de las tablas

In [4]:
tabla_tiempo = '''
CREATE TABLE IF NOT EXISTS `clima2`.`tiempo` (
  `idtiempo` INT NOT NULL AUTO_INCREMENT,
  `timepoint`INT NOT NULL,
  `fecha` DATE NOT NULL,
  PRIMARY KEY (`idtiempo`))
ENGINE = InnoDB;
'''

tabla_localidades = '''
CREATE TABLE IF NOT EXISTS `clima2`.`localidades` (
  `idlocalidad` INT NOT NULL AUTO_INCREMENT,
  `latitud` DECIMAL(6,4) NOT NULL,
  `longitud` DECIMAL(6,4) NOT NULL,
  `ciudad` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`idlocalidad`))
ENGINE = InnoDB;
'''

tabla_mediciones = '''
CREATE TABLE IF NOT EXISTS `clima2`.`mediciones` (
  `idmedicion` INT NOT NULL AUTO_INCREMENT,
  `cloudcover` INT NOT NULL,
  `lifted_index` INT NOT NULL,
  `prec_type` VARCHAR(10) NOT NULL,
  `prec_amount` INT NOT NULL,
  `temp2m` INT NOT NULL,
  `rh2m` INT NOT NULL,
  `weather` VARCHAR(15) NOT NULL,
  `wind_direction` VARCHAR(15) NOT NULL,
  `wind_speed` INT NOT NULL,
  `seeing` DECIMAL NOT NULL,
  `transparency` INT NOT NULL,
  `idtiempo` INT NOT NULL,
  `idlocalidad` INT NOT NULL,
  PRIMARY KEY ( `idmedicion`, `idtiempo`, `idlocalidad`),
  INDEX `fk_mediciones_tiempo1_idx` (`idtiempo` ASC) ,
  INDEX `fk_mediciones_localidades1_idx` (`idlocalidad` ASC) ,
  CONSTRAINT `fk_mediciones_tiempo1`
    FOREIGN KEY (`idtiempo`)
    REFERENCES `clima2`.`tiempo` (`idtiempo`),
  CONSTRAINT `fk_mediciones_localidades1`
    FOREIGN KEY (`idlocalidad`)
    REFERENCES `clima2`.`localidades` (`idlocalidad`))
ENGINE = InnoDB;

'''

## Llamamos a la Clase

In [5]:
carga = Cargar("clima2", "admin")

In [6]:
carga

### Creamos la BBDD 

In [7]:
carga.crear_bbdd()

Conexión realizada con éxito


### Creamos las tablas 

In [8]:
carga.crear_insertar_tabla(tabla_tiempo)
carga.crear_insertar_tabla(tabla_localidades)
carga.crear_insertar_tabla(tabla_mediciones)

### Insertamos los datos

📌 Dado que nuestra BBDD de datos esta vacía las funciones de sacar id y fecha nos debe devolver que no encuentra los ids

In [9]:
id_ciudad = carga.sacar_id_ciudad("Madrid")
id_ciudad

'Sorry, no tenemos esa ciudad en la BBDD y por lo tanto no te podemos dar su id. '

In [10]:
id_fecha = carga.sacar_id_fecha(3, "2022-03-16")
id_fecha

'Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. '

In [11]:
# tabla tiempo

for indice, fila in df.iterrows(): 
    
    query_tiempo = f"""
            INSERT INTO tiempo (timepoint, fecha) # como la primary key la hemos puesto como autoincremental no hace falta introducirla en el insert
            VALUES ( "{fila["timepoint"]}", "{fila['fecha']}");
            """
    carga.crear_insertar_tabla(query_tiempo)

In [15]:
# tabla localidades

for indice, fila in df.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_localidad = f"""
                INSERT INTO localidades (latitud, longitud, ciudad) 
                VALUES ( "{fila["latitud"]}", "{fila['longitud']}", "{fila['ciudad']}");
                """
    # llamamos a la funcion check_ciudades para saber que ciudades tenemos en la BBDD
    ciudades = carga.check_ciudades()
    
    # Si la BBDD de datos esta vacía,  podemos insertar la ciudad. 
    # 
    if len(ciudades) == 0 or fila['ciudad'] not in ciudades[0]: 
        carga.crear_insertar_tabla( query_localidad)

    else:
        print(f"{fila['ciudad']} ya esta en nuestra BBDD")

Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid ya esta en nuestra BBDD
Madrid y

In [16]:
# tabla mediciones

for indice, fila in df.iterrows():
    
    id_ciudad = carga.sacar_id_ciudad(fila['ciudad'])
    id_fecha = carga.sacar_id_fecha( fila["timepoint"], fila["fecha"])
    
    query_medidas = f"""
                INSERT INTO mediciones (cloudcover, lifted_index, prec_type, prec_amount, temp2m, rh2m, weather, wind_direction, wind_speed, seeing, transparency, idtiempo, idlocalidad) 
                VALUES ({fila["cloudcover"]}, {fila['lifted_index']}, "{fila['prec_type']}", {fila['prec_amount']},{fila['temp2m']},{fila['rh2m']}, "{fila['weather']}","{fila['wind10m.direction']}",{fila['wind10m.speed']}, {fila['seeing']}, {fila['transparency']}, {id_fecha}, {id_ciudad});
                """
    carga.crear_insertar_tabla( query_medidas)
    

Y voilá! Ya tenemos nuestros datos en nuestra BBDD listos para que queden guardados para siempre!!!! 😀